In [1]:
################################                        
# Finding angle between 
# GRB position
# and NaI, BGO detectors pointing directions
################################

from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from astropy.io import ascii
import time
import progressbar
import os
import math
from astropy.io import fits

In [2]:
path = 'bn080723913/current/'    # path to current dataset


datafiles = os.listdir(path)     # list the files in dataset dir
#print(datafiles)

In [3]:
# GBM detectors
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb', 'b0', 'b1'])

In [4]:
# List Rsponse files in the current directory
rspfiles = []

for file in datafiles:
    if 'cspec' in file and 'rsp' in file:
        rspfiles.append(file)
        
        
    

In [11]:
ifl = rspfiles[0]
#print(ifl)

path1 = os.path.join(path, ifl)
#print(path1)

with fits.open(path1) as ifits:
    #print(ifits.info())
    try:
        RA_OBJ = ifits[0].header['RA_OBJ']
        DEC_OBJ = ifits[0].header['DEC_OBJ']
    except:
        try:
            RA_OBJ = ifits[1].header['RA_OBJ']
            DEC_OBJ = ifits[1].header['DEC_OBJ']
        except:
            try:
                RA_OBJ = ifits[2].header['RA_OBJ']
                DEC_OBJ = ifits[2].header['DEC_OBJ']
            except:
                RA_OBJ = 'not available'
                DEC_OBJ = 'not available'


print("RA Object:", RA_OBJ)
print("Dec Object:", DEC_OBJ)


# The Spacecraft pointing directions for a triggered dataset are in a file with string 'trigdat' in its name
trigdat_file = []

for file in datafiles:
    if 'trigdat' in file:
        trigdat_file.append(file)

if len(trigdat_file) > 0:
    print("Found trigdat file", trigdat_file)
else:
    print("trigdat not found, check if the data is a triggered dataset and the file is present")




#Read pointing direction info from trigdat files
anglef = fits.open(os.path.join(path, trigdat_file[0]))
# Get spacecraft pointing
RA_SCX = anglef[0].header['RA_SCX']
DEC_SCX = anglef[0].header['DEC_SCX']
RA_SCZ = anglef[0].header['RA_SCZ']
DEC_SCZ = anglef[0].header['DEC_SCZ']
anglef.close()


#print(RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ)     

RA Object: 113.3
Dec Object: -19.7
Found trigdat file ['glg_trigdat_all_bn080723913_v01.fit']


In [6]:
#The following code is part of 
#https://github.com/giacomov/gtburst/blob/master/python/GtBurst/angularDistance.py


#Angles of NaI detectors in spacecraft coordinates
#(from Meegan et al.)
#[zenith, azimuth]
DetDir        = {}
DetDir['n0']  = [20.58, 45.89]
DetDir['n1']  = [45.31, 45.11]
DetDir['n2']  = [90.21, 58.44]
DetDir['n3']  = [45.24, 314.87]
DetDir['n4']  = [90.27, 303.15]
DetDir['n5']  = [89.79, 3.35]
DetDir['n6']  = [20.43, 224.93]
DetDir['n7']  = [46.18, 224.62]
DetDir['n8']  = [89.97, 236.61]
DetDir['n9']  = [45.55, 135.19]
DetDir['na']  = [90.42, 123.73]
DetDir['nb']  = [90.32, 183.74]
#Angles of BGO detectors, just to mean they are in two opposite directions     
DetDir['b0']  = [90.0, 0.00]
DetDir['b1']  = [90.0, 180.00]
#Of course the LAT is at 0,0
DetDir['LAT-LLE'] = [0.0,0.0]
DetDir['LAT'] = [0.0,0.0]




def getDetectorAngle(ra_scx,dec_scx,ra_scz,dec_scz,sourceRa,sourceDec,detector):
  if detector in DetDir.keys():
    t                         = DetDir[detector][0]
    p                         = DetDir[detector][1]
    ra,dec                    = getRaDec(ra_scx,dec_scx,ra_scz,dec_scz,t,p)
    return getAngularDistance(sourceRa,sourceDec,ra,dec)
  else:
    raise ValueError('Detector %s is not recognized' %(detector))        
pass


def getAngularDistance(ra1, dec1, ra2, dec2):
    
    # Vincenty formula, stable also at antipodes
    
    lon1 = np.deg2rad(ra1)
    lat1 = np.deg2rad(dec1)
    lon2 = np.deg2rad(ra2)
    lat2 = np.deg2rad(dec2)
        
    sdlon = np.sin(lon2 - lon1)
    cdlon = np.cos(lon2 - lon1)
    slat1 = np.sin(lat1)
    slat2 = np.sin(lat2)
    clat1 = np.cos(lat1)
    clat2 = np.cos(lat2)

    num1 = clat2 * sdlon
    num2 = clat1 * slat2 - slat1 * clat2 * cdlon
    denominator = slat1 * slat2 + clat1 * clat2 * cdlon

    return np.rad2deg(np.arctan2(np.sqrt(num1 ** 2 + num2 ** 2), denominator))

def getAngularDistance_old(ra1,dec1,ra2,dec2):

  dlat = np.deg2rad(dec2 - dec1)
  dlon = np.deg2rad(ra2 - ra1)
  dec1 = np.deg2rad(dec1)
  dec2 = np.deg2rad(dec2)
  a = np.sin(dlat/2.)*np.sin(dlat/2.) + np.cos(dec1)*np.cos(dec2)*np.sin(dlon/2.)*np.sin(dlon/2.)
  
  c  = 2*np.arctan2(np.sqrt(a), np.sqrt(1.-a))
  
  return np.rad2deg(c)

def _getNaIDirection(self,detectors):
  DetDir  = {}
  outDetDir     = {}
  for det in detectors:
    outDetDir[det] = DetDir[det] 
  pass
  return outDetDir
pass

def getThetaPhi(ra_scx,dec_scx,ra_scz,dec_scz,RA,DEC):
  v0                          = getVector(RA,DEC)
  vx                          = getVector(ra_scx,dec_scx)
  vz                          = getVector(ra_scz,dec_scz)
  vy                          = Vector(vz.cross(vx))
  
  theta                       = math.degrees(v0.angle(vz))    
  phi                         = math.degrees(math.atan2(vy.dot(v0),vx.dot(v0)))
  if phi<0: phi+=360
  return theta, phi

def getRaDec(ra_scx,dec_scx,ra_scz,dec_scz,theta,phi):
  vx                          = getVector(ra_scx,dec_scx)
  vz                          = getVector(ra_scz,dec_scz)
  
  vxx                         = Vector(vx.rotate(phi,vz))  
  vy                          = Vector(vz.cross(vxx))
  
  vzz                         = vz.rotate(theta,vy)
   
  ra                          = math.degrees(math.atan2(vzz[1],vzz[0]))
  dec                         = math.degrees(math.asin(vzz[2]))
  
  if(ra<0):
    ra                       += 360.0
  return ra,dec  
pass

def getVector(ra,dec):
  ra1                         = math.radians(ra)
  dec1                        = math.radians(dec)
  
  cd                          = math.cos(dec1)
  
  return Vector([math.cos(ra1) * cd, 
                          math.sin(ra1) * cd,
                          math.sin(dec1)])

class Vector(object):
  def __init__(self,array):
    self.vector               = np.array(array)
  pass
  
  def rotate(self,angle,axisVector):
    ang                       = math.radians(angle)
    matrix                    = self._getRotationMatrix(axisVector.vector,ang)
    #print matrix
    return np.dot(matrix,self.vector)
  
  def cross(self,vector):
    return np.cross(self.vector,vector.vector)
    
  def _getRotationMatrix(self,axis,theta):
    axis                      = axis/np.sqrt(np.dot(axis,axis))
    a                         = np.cos(theta/2)
    b,c,d                     = -axis*np.sin(theta/2)
    return np.array([[a*a+b*b-c*c-d*d, 2*(b*c+a*d),2*(b*d-a*c)],
                     [2*(b*c-a*d), a*a+c*c-b*b-d*d, 2*(c*d+a*b)],
                     [2*(b*d+a*c), 2*(c*d-a*b), a*a+d*d-b*b-c*c]])           
   
  def norm(self):
     return np.linalg.norm(self.vector)
  
  def dot(self,vector):
    return np.dot(self.vector,vector.vector)
   
  def angle(self,vector):
     return math.acos(np.dot(self.vector,vector.vector)/(self.norm()*vector.norm()))
pass

In [12]:

#Calculate the angles using Vianello's code
angle = np.array([getDetectorAngle(RA_SCX, DEC_SCX, RA_SCZ, DEC_SCZ, RA_OBJ, DEC_OBJ,
                                                             detectors[i]) for i in range(14)])
angleString = ['%3.0f' % (angle[i]) for i in range(13)]
angleToGRB = angle
    

In [15]:
# List angles based on sorted angles (degree)

#print("Angles", angle)
angle_s = np.argsort(angle)
#print(angle_s)
prioritydetectors = detectors[angle_s]
priorityangles = angle[angle_s]

print("GBM detectors with angles to the Source\n")
for i in range(14):
    print("Detector: %s Angle %0.3f"%(prioritydetectors[i], priorityangles[i]))


GBM detectors with angles to the Source

Detector: n0 Angle 5.248
Detector: n1 Angle 25.289
Detector: n6 Angle 41.170
Detector: n3 Angle 44.139
Detector: n9 Angle 53.750
Detector: n7 Angle 66.789
Detector: n5 Angle 71.264
Detector: n2 Angle 71.590
Detector: b0 Angle 72.094
Detector: n4 Angle 89.532
Detector: na Angle 91.320
Detector: b1 Angle 107.906
Detector: n8 Angle 108.896
Detector: nb Angle 108.912


In [24]:
pds = prioritydetectors[priorityangles<60]
#print(pds)

detectors_set1 = ['n0', 'n1', 'n2', 'n3', 'n4', 'n5']
detectors_set2 = ['n6', 'n7', 'n8', 'n9', 'na', 'nb']

dets_till_n5 = []
dets_from_n5_to_nb = []
for det in pds:
    if det in detectors_set1:
        dets_till_n5.append(det)
        
    elif det in detectors_set2:
        dets_from_n5_to_nb.append(det)
        
        
#print(dets_till_n5)
#print(dets_from_n5_to_nb)

#Select BGO

d1 = len(dets_till_n5)
d2 = len(dets_from_n5_to_nb)


selected_detectors = []

if d1 > d2:
    selected_detectors.append(dets_till_n5)
    selected_detectors.append('b0')
elif d2 < d1:
    selected_detectors.append(dets_from_n5_to_nb)
    selected_detectors.append('b1')
else:
    selected_detectors.append(pds)
    selected_detectors.append('b0')
    selected_detectors.append('b1')
    
NaI_dets = selected_detectors[0]
BGO_dets = selected_detectors[1]

print("Detectors selected \n NaI: %s \n BGO: %s"%(NaI_dets, BGO_dets))  



Detectors selected 
 NaI: ['n0', 'n1', 'n3'] 
 BGO: b0
